In [1]:
!pip install --upgrade kiteconnect
!pip install python-dotenv
!pip install python-dateutil

Requirement already up-to-date: kiteconnect in c:\users\nisha\miniconda3\lib\site-packages (3.9.4)


In [2]:
import logging
import os
from kiteconnect import KiteConnect
from dotenv import load_dotenv
load_dotenv()
import bs_threading
import importlib
import time
from datetime import datetime, timedelta
import dateutil
import pandas as pd
importlib.reload(bs_threading)
from bs_threading import bs_threadify, bs_make_throttle_ready_func


In [3]:
KITE_CONNECT_API_KEY = os.getenv("KITE_CONNECT_API_KEY")
KITE_CONNECT_API_SECRET = os.getenv("KITE_CONNECT_API_SECRET")

In [4]:

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key=KITE_CONNECT_API_KEY)



### Get the url based on your api key, which will have request token, this is in order to verify the app itself(bsstonks) is valid and it'll get redirected to the callback url

In [5]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=7e6nr4sfi674uleu&v=3


### Given we have request token and redirected to our callback with it, our app is valid, but is the app in the hands/server of the right person or not? only API secret can verify that. 
### We'll pass that api secret with the request token to create a session which give us Access_token, to the rightful person that we are

In [6]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

kite_session_data = kite.generate_session("F870mmDGJSBywiITCaL4cSy8Yr0I4c0c", api_secret=KITE_CONNECT_API_SECRET)
print("THE TOKEN:",kite_session_data["access_token"])
kite.set_access_token(kite_session_data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


THE TOKEN: 0YXSASbwBNyIaobrolnvXrRzYPmx9f7z


In [8]:
kite.set_access_token("0YXSASbwBNyIaobrolnvXrRzYPmx9f7z")

In [9]:
kite.orders()
kite.holdings()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /orders HTTP/1.1" 200 30
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None


[{'tradingsymbol': 'BHEL',
  'exchange': 'BSE',
  'instrument_token': 128026372,
  'isin': 'INE257A01026',
  'product': 'CNC',
  'price': 0,
  'quantity': 250,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 250,
  'authorised_quantity': 0,
  'authorised_date': '2021-07-11 00:00:00',
  'opening_quantity': 250,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average_price': 67.9,
  'last_price': 66.45,
  'close_price': 66.45,
  'pnl': -362.5000000000007,
  'day_change': 0,
  'day_change_percentage': 0},
 {'tradingsymbol': 'MANGALAM',
  'exchange': 'BSE',
  'instrument_token': 136355076,
  'isin': 'INE584F01014',
  'product': 'CNC',
  'price': 0,
  'quantity': 1,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 1,
  'authorised_quantity': 0,
  'authorised_date': '2021-07-11 00:00:00',
  'opening_quantity': 1,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average_price': 132.85,
  'last_pric

In [10]:
all_instruments = kite.instruments()
all_instruments_nse = kite.instruments(exchange="NSE")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 1100280
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NSE HTTP/1.1" 200 60569


In [11]:
df_all_instruments = pd.DataFrame(all_instruments)
os.makedirs("input/kite_instruments", exist_ok=True)
df_all_instruments.to_csv("input/kite_instruments/instrument_list.csv", index=False)
df_all_instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83227,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83228,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83229,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83230,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,,0.00,0.0500,1,EQ,NSE,NSE


In [12]:
df_all_instruments["segment"].unique()

array(['BCD-OPT', 'BCD-FUT', 'BCD', 'BSE', 'INDICES', 'CDS-OPT',
       'CDS-FUT', 'MCX-FUT', 'MCX-OPT', 'NFO-OPT', 'NFO-FUT', 'NSE'],
      dtype=object)

In [13]:

print("Total Instruments:", len(all_instruments))
print("Total EQ",len([x for x in all_instruments if x['instrument_type']=="EQ"]))
print("Sample instsrument:", all_instruments[0])
all_tcs_match = [x for x in all_instruments if "tcs" in x['tradingsymbol'].lower()]

Total Instruments: 83232
Total EQ 16863
Sample instsrument: OrderedDict([('instrument_token', 556726790), ('exchange_token', '2174714'), ('tradingsymbol', 'EURINR21AUG84.5000CE'), ('name', 'EURINR'), ('last_price', 0.0), ('expiry', datetime.date(2021, 8, 27)), ('strike', 84.5), ('tick_size', 0.0025), ('lot_size', 1), ('instrument_type', 'CE'), ('segment', 'BCD-OPT'), ('exchange', 'BCD')])


In [14]:
print("Total TCS match:", len(all_tcs_match))
for x in all_tcs_match:
    print(x['tradingsymbol'], x['name'], "InstrumentType:", x['instrument_type'],'Exchange:',x['exchange'], "InstrumentToken:", x['instrument_token'])
all_tcs_match

Total TCS match: 433
TCS TATA CONSULTANCY SERVICES. InstrumentType: EQ Exchange: BSE InstrumentToken: 136330244
WSTCSTPAPR WEST COAST PAPER MILLS  InstrumentType: EQ Exchange: BSE InstrumentToken: 128113668
TCS21JUL2400CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 27960322
TCS21JUL2400PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 27960578
TCS21JUL2450CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 27960834
TCS21JUL2450PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 27961090
TCS21JUL2500CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 27961346
TCS21JUL2500PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 27961602
TCS21JUL2550CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 27961858
TCS21JUL2550PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 27962114
TCS21JUL2600CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 27962370
TCS21JUL2600PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 27962626
TCS21JUL2650C

TCS21SEP3880CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 31128834
TCS21SEP3880PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 31129090
TCS21SEP3900CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 31129346
TCS21SEP3900PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 31129602
TCS21SEP3920CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 31129858
TCS21SEP3920PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 31130114
TCS21SEP3940CE TCS InstrumentType: CE Exchange: NFO InstrumentToken: 31130370
TCS21SEP3940PE TCS InstrumentType: PE Exchange: NFO InstrumentToken: 31130626
TCS21AUGFUT TCS InstrumentType: FUT Exchange: NFO InstrumentToken: 12876290
TCS21JULFUT TCS InstrumentType: FUT Exchange: NFO InstrumentToken: 13657602
TCS21SEPFUT TCS InstrumentType: FUT Exchange: NFO InstrumentToken: 12568834
TCS TATA CONSULTANCY SERV LT InstrumentType: EQ Exchange: NSE InstrumentToken: 2953217
WSTCSTPAPR WEST COAST PAPER MILLS LT InstrumentType: EQ Excha

[OrderedDict([('instrument_token', 136330244),
              ('exchange_token', '532540'),
              ('tradingsymbol', 'TCS'),
              ('name', 'TATA CONSULTANCY SERVICES.'),
              ('last_price', 0.0),
              ('expiry', ''),
              ('strike', 0.0),
              ('tick_size', 0.05),
              ('lot_size', 1),
              ('instrument_type', 'EQ'),
              ('segment', 'BSE'),
              ('exchange', 'BSE')]),
 OrderedDict([('instrument_token', 128113668),
              ('exchange_token', '500444'),
              ('tradingsymbol', 'WSTCSTPAPR'),
              ('name', 'WEST COAST PAPER MILLS '),
              ('last_price', 0.0),
              ('expiry', ''),
              ('strike', 0.0),
              ('tick_size', 0.05),
              ('lot_size', 1),
              ('instrument_type', 'EQ'),
              ('segment', 'BSE'),
              ('exchange', 'BSE')]),
 OrderedDict([('instrument_token', 27960322),
              ('exchange_token',

In [15]:
kite.quote(["2953217"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=2953217 HTTP/1.1" 200 None


{'2953217': {'instrument_token': 2953217,
  'timestamp': datetime.datetime(2021, 7, 9, 19, 2, 20),
  'last_trade_time': datetime.datetime(2021, 7, 9, 15, 59, 51),
  'last_price': 3208.15,
  'last_quantity': 0,
  'buy_quantity': 0,
  'sell_quantity': 0,
  'volume': 0,
  'average_price': 0,
  'oi': 0,
  'oi_day_high': 0,
  'oi_day_low': 0,
  'net_change': 0,
  'lower_circuit_limit': 2887.35,
  'upper_circuit_limit': 3528.95,
  'ohlc': {'open': 3250, 'high': 3282, 'low': 3201, 'close': 3258.5},
  'depth': {'buy': [{'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0}],
   'sell': [{'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0}]}}}

In [16]:
#TCS NSE TOKEN: 2953217
# day : 2000 days  = 2000 entries
# 60minute : 400 days = 400 * 24 = 9600 entries
# 30minute : 200 days = 
# 15minute : 200 days =
# 5minute : 100 days = 
# 3minute : 100 days =
# minute : 60 days = 
interval_data_span_limit = {
    'day': 2000,
    '60minute':400,
    '30minute':200,
    '15minute':200,
    '5minute':100,
    '3minute':100,
    'minute':60,
}
kite.historical_data(2953217, "2021-06-15", "2021-06-16", "day")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2953217/day?from=2021-06-15&to=2021-06-16&interval=day&continuous=0&oi=0 HTTP/1.1" 200 None


[{'date': datetime.datetime(2021, 6, 15, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 3298,
  'high': 3298,
  'low': 3251.55,
  'close': 3262.75,
  'volume': 1304985},
 {'date': datetime.datetime(2021, 6, 16, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 3262.1,
  'high': 3294.7,
  'low': 3253,
  'close': 3274.35,
  'volume': 1635552}]

In [17]:
kite.historical_data(2953217, "2021-06-15", "2021-06-16", "hour", oi=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2953217/hour?from=2021-06-15&to=2021-06-16&interval=hour&continuous=0&oi=1 HTTP/1.1" 200 None


[{'date': datetime.datetime(2021, 6, 15, 9, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3298,
  'high': 3298,
  'low': 3278,
  'close': 3281.75,
  'volume': 309040,
  'oi': 0},
 {'date': datetime.datetime(2021, 6, 15, 10, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3282,
  'high': 3286.95,
  'low': 3277,
  'close': 3278.45,
  'volume': 148314,
  'oi': 0},
 {'date': datetime.datetime(2021, 6, 15, 11, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3278.45,
  'high': 3284.5,
  'low': 3268.6,
  'close': 3273,
  'volume': 129300,
  'oi': 0},
 {'date': datetime.datetime(2021, 6, 15, 12, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3273.2,
  'high': 3278,
  'low': 3270,
  'close': 3275.5,
  'volume': 157827,
  'oi': 0},
 {'date': datetime.datetime(2021, 6, 15, 13, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3275.25,
  'high': 3276.5,
  'low': 3269.1,
  'close': 3272,
  'volume': 142939,
  'oi': 0},
 {'date': datetime.datetime(2021, 6, 15, 14, 15, tzinfo=tzoffset(None, 19800)),
  'open': 3272

### We want to keep 3 request per second limit of zerodha
- ### we'll use multiple threads
- ### we'll use the shared throttle function which only returns true if we haven't made request in last x(can be sub second) seconds

In [17]:
throttle_ready_func = bs_make_throttle_ready_func(min_interval_second=1.0/3.0)
def worker_func(data):
    while not throttle_ready_func():
        time.sleep(0.001)
    print("this is the real life")

worker_data = [0] * 6
last_time = time.time()
queue = bs_threadify(worker_data, worker_func, num_threads=16)
print("Time took", time.time()-last_time)



this is the real life
this is the real life
this is the real life
this is the real life
this is the real life
this is the real life
Time took 2.0556788444519043


### Okay then throttle seems to be working with multithreading, let's create the data fetcher

In [18]:
all_instruments_nse_eq = [x for x in all_instruments_nse if x['instrument_type']=="EQ"]
print(len(all_instruments_nse),len(all_instruments_nse_eq))
all_instruments_nse_eq[0]

3835 3835


OrderedDict([('instrument_token', 264713),
             ('exchange_token', '1034'),
             ('tradingsymbol', 'HANGSENG BEES-NAV'),
             ('name', 'HANGSENG BEES-NAV'),
             ('last_price', 0.0),
             ('expiry', ''),
             ('strike', 0.0),
             ('tick_size', 0.0),
             ('lot_size', 0),
             ('instrument_type', 'EQ'),
             ('segment', 'INDICES'),
             ('exchange', 'NSE')])

In [30]:
def kite_instrument_to_filename(instrument_dict, interval="day", expiry="1mo"):
    kite_instrument_token = instrument_dict['instrument_token']
    trading_symbol = instrument_dict['tradingsymbol']
    instrument_type = instrument_dict['instrument_type']
    segment = instrument_dict['segment']
    name = instrument_dict['name']
    exchange = instrument_dict['exchange']
    if exchange == "NFO" and (instrument_type == "CE" or instrument_type == "PE"):
        trading_symbol = trading_symbol.replace(name,"")
        trading_symbol = trading_symbol[5:]
        trading_symbol = name + trading_symbol
        trading_symbol += expiry
    # we've tested that _ is valid seperator by evaluating for all the instruments (86k)
    result_file = f"{kite_instrument_token}_{trading_symbol}_{instrument_type}_{segment}_{exchange}_{interval}.csv"
    return result_file

In [32]:
kite_instrument_to_filename([x for x in all_instruments if x['segment'] == "NFO-OPT" and x['name'].startswith("REL")][0])

'27405826_RELIANCE1360CE1mo_CE_NFO-OPT_NFO_day.csv'

In [19]:


def make_sync_single_symbol_worker_func(throttle_ready_func, output_path = None, expiry = None):
    #NOTE: input to our stock prediction/training system
    if output_path is None:
        output_path = "input/kite_historical"
    os.makedirs(output_path, exist_ok=True)
    def sync_single_symbol_worker_func(data):
        nonlocal throttle_ready_func
        nonlocal expiry
        nonlocal output_path
        # data['instrument_dict'] this should be instrument dictionary that zerodha returns
        # data['interval'] 
        
        data_interval = data['interval']
        if 'fetch_past' not in data:
            data['fetch_past'] = True
            
        interval_span_days = interval_data_span_limit[data_interval]
        # we've tested that _ is valid seperator by evaluating for all the instruments (86k)
        result_file = kite_instrument_to_filename(data['instrument_dict'], interval=data['interval'], expiry=expiry)
        result_file_path = os.path.join(output_path,result_file)
        df_to_sync = None
        if not os.path.exists(result_file_path):
            df_to_sync = pd.DataFrame([],columns=["date","open","high","low","close","volume"])
#             df_to_sync.to_csv(result_file_path, index=False, header=True)
        else:
            df_to_sync = pd.read_csv(result_file_path,index_col=False)



        #     
        # Past direction
        while True and (data['fetch_past'] or df_to_sync.shape[0]==0):

            #IMPORTANT: THROTTLE LIMIT MUST BE RESPECTED
            while not throttle_ready_func():
                time.sleep(0.001)

            end = datetime.now()
            start = end - timedelta(days=interval_span_days-1)

            if df_to_sync.shape[0] > 0:
                end = df_to_sync.iloc[0]["date"]
                if type(end) == str:
                    end = dateutil.parser.parse(end)
                end = end - timedelta(days=1)
                #dates are inclusive so start date (historical date) needs to be adjusted
                start = end - timedelta(days=interval_span_days-1)

            historical_data = kite.historical_data(kite_instrument_token, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"), data_interval, oi=True)
            for x in historical_data:
                x['date'] = str(x['date'])

            # WHEN API RETURNS NO DATA; EXIT.
            if len(historical_data) == 0:
                break


            df_to_sync_more_hist = pd.DataFrame(historical_data)

            df_to_sync = df_to_sync_more_hist.append(df_to_sync, ignore_index=True)      
            df_to_sync.to_csv(result_file_path, index=False)

            # if it returns less than possible working days of data then just 
            if len(historical_data) < 3 or dateutil.parser.parse(historical_data[-1]['date']) - dateutil.parser.parse(historical_data[0]['date']) < timedelta(days=int(interval_span_days*0.5 - 15)):
                break


        #     
        # Future direction
        while True:

            #IMPORTANT: THROTTLE LIMIT MUST BE RESPECTED
            while not throttle_ready_func():
                time.sleep(0.001)

            start = datetime.now()
            end = start + timedelta(days=interval_span_days-1)

            if df_to_sync.shape[0] > 0:
                start = df_to_sync.iloc[-1]["date"]
                if type(start) == str:
                    start = dateutil.parser.parse(start)
                # what if we run it while the stock market is open, and run it again after it closes
                # today's data must be overwritten/appended
                start = start - timedelta(days=1)
                #dates are inclusive so start date (historical date) needs to be adjusted
                end = start + timedelta(days=interval_span_days-1)


            historical_data = kite.historical_data(kite_instrument_token, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"), data_interval, oi=True)
            
            for x in historical_data:
                x['date'] = str(x['date'])

            if len(historical_data) == 0:
                break
            # WHEN API RETURNS NO DATA; EXIT.


            df_to_sync_more_hist = pd.DataFrame(historical_data)
            # rewrite last few days of new data onto old data; this takes care of running it during trdaing window
            overlaps_count = df_to_sync[df_to_sync['date']>=str(historical_data[0]['date'])].shape[0]
            

            df_to_sync = df_to_sync.iloc[:-overlaps_count]
            
            df_to_sync = df_to_sync.append(df_to_sync_more_hist, ignore_index=True)      
            df_to_sync.to_csv(result_file_path, index=False)

            # if it returns less than possible working days of data then just 
            if len(historical_data) < 3 or dateutil.parser.parse(historical_data[-1]['date']) - dateutil.parser.parse(historical_data[0]['date']) < timedelta(days=int(interval_span_days*0.5 - 15)):
                break

    return sync_single_symbol_worker_func
# USage:

importlib.reload(bs_threading)
sync_single_symbol_worker_func = make_sync_single_symbol_worker_func(bs_make_throttle_ready_func(min_interval_second=1.01/3.0))
sync_single_symbol_worker_func({'instrument_dict':all_tcs_match[0],'interval':'day'})

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/136330244/day?from=2002-07-11&to=2007-12-31&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/136330244/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None


### Yay! we made it, we made the perfect sync code!!!

#### Let's do day wise sync

In [20]:
def sync_instruments(instuments_list, interval="15minute", need_daily_exist=False, fetch_past=True):
    should_terminate=False

    sync_single_symbol_worker_func = make_sync_single_symbol_worker_func(bs_make_throttle_ready_func(min_interval_second=1.01/3.0))
    output_path = "input/kite_historical"
    os.makedirs(output_path, exist_ok=True)
    symbol_worker_data = []
    for instrument_dict in instuments_list:
        
        


        # we've tested that _ is valid seperator by evaluating for all the instruments (86k)
        result_file = f"{kite_instrument_token}_{trading_symbol}_{instrument_type}_{segment}_{exchange}_day.csv"
        result_file_path = os.path.join(output_path,result_file)
        if os.path.exists(result_file_path) or not need_daily_exist:
            symbol_worker_data.append({'instrument_dict':instrument_dict,'interval':interval,'fetch_past':fetch_past})
#             sync_single_symbol_worker_func({'instrument_dict':instrument_dict,'interval':interval})

#     print(len(symbol_worker_data), " out of ", len(all_instruments_nse))
    def should_terminate_func():
        nonlocal should_terminate
        return should_terminate
    
    bs_threadify(symbol_worker_data, sync_single_symbol_worker_func, num_threads=8, should_terminate_func=should_terminate_func)
    should_terminate = True

In [21]:
# sync_single_symbol_worker_func = make_sync_single_symbol_worker_func(bs_make_throttle_ready_func(min_interval_second=1.01/3.0))

# symbol_worker_data = []
# for instrument_dict in all_instruments_nse:
#     symbol_worker_data.append({'instrument_dict':instrument_dict,'interval':'day'})
# bs_threadify(symbol_worker_data, sync_single_symbol_worker_func, num_threads=8)
sync_instruments(all_instruments_nse, interval='day',fetch_past=False)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/264969/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/264457/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/268041/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/265993/day?from=2021-06-29&to=2026-12-19&interval=day&c

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/263689/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/261129/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/262921/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/264713/day?from=2021-06-29&to=2026-12-19&interval=day&c

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/537601/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/1749249/day?from=2021-06-17&to=2026-12-07&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/489985/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/957441/day?from=2016-01-17&to=2021-07-08&interval=day&cont

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/940545/day?from=2021-07-08&to=2026-12-28&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/620033/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/1255169/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/1255169/day?from=2021-07-08&to=2026-12-28&interval=day&conti

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/5716481/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/1537/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/692737/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/5710593/day?from=2016-01-17&to=2021-07-08&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connect

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/3945985/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/269569/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2886401/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/5011969/day?from=2021-06-29&to=2026-12-19&interval=da

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2691841/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2347009/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/6194177/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/594433/day?from=2021-06-20&to=2026-12-10&interval=da

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/944129/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/84481/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/941057/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/945665/day?from=2021-06-29&to=2026-12-19&interval=day&co

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/4330241/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/7496705/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/3601153/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/958721/day?from=2021-06-29&to=2026-12-19&interval=da

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/3050241/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/768513/day?from=2021-06-29&to=2026-12-19&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/154113/day?from=2021-06-27&to=2026-12-17&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/275457/day?from=2021-06-29&to=2026-12-19&interval=day&

#### There are some useless symbols there with no data, only do sub day interval for those for which the day data exist

In [13]:
# sync_instruments(all_instruments_nse, interval='15minute', need_daily_exist=True, fetch_past=False)

In [12]:
# sync_instruments(all_instruments_nse, interval='5minute', need_daily_exist=True, fetch_past=True)


In [70]:
# historical_data = kite.historical_data(264713, "2021-06-17", "2022-01-02", "15minute", oi=True)
# pd.DataFrame(historical_data)
nifty_50_instruments = [x for x in all_instruments_nse if "nifty 50" in x['tradingsymbol'].lower()]

nifty_50_instruments


[OrderedDict([('instrument_token', 256265),
              ('exchange_token', '1001'),
              ('tradingsymbol', 'NIFTY 50'),
              ('name', 'NIFTY 50'),
              ('last_price', 0.0),
              ('expiry', ''),
              ('strike', 0.0),
              ('tick_size', 0.0),
              ('lot_size', 0),
              ('instrument_type', 'EQ'),
              ('segment', 'INDICES'),
              ('exchange', 'NSE')]),
 OrderedDict([('instrument_token', 268041),
              ('exchange_token', '1047'),
              ('tradingsymbol', 'NIFTY 500'),
              ('name', 'NIFTY 500'),
              ('last_price', 0.0),
              ('expiry', ''),
              ('strike', 0.0),
              ('tick_size', 0.0),
              ('lot_size', 0),
              ('instrument_type', 'EQ'),
              ('segment', 'INDICES'),
              ('exchange', 'NSE')])]

In [79]:
# all_instruments_nse_eq[all_instruments_nse_eq["tradingsymbol"].str.lower().str.startswith("nifty") & all_instruments_nse_eq["name"].str.lower().str.startswith("nifty")]
print("Total instruments to fetch:", len(nifty_50_instruments))
sync_instruments(nifty_50_instruments, interval="day")

Total instruments to fetch: 2


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/256265/day?from=1990-07-11&to=1995-12-31&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/256265/day?from=2021-06-21&to=2026-12-11&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/268041/day?from=1993-12-16&to=1999-06-07&interval=day&continuous=0&oi=1 HTTP/1.1" 200 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/268041/day?from=2021-06-21&to=2026-12-11&interval=day&conti

In [58]:
historical_data = kite.historical_data(136330244, "2015-04-01", "2015-05-10", "day", oi=True)
        
df_to_sync_more_hist = pd.DataFrame(historical_data)
df_to_sync_more_hist2 = pd.DataFrame(historical_data)
df_to_sync_more_hist2

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/136330244/day?from=2015-04-01&to=2015-05-10&interval=day&continuous=0&oi=1 HTTP/1.1" 200 None


,date,open,high,low,close,volume,oi
0,2015-04-01 00:00:00+05:30,2565.00,2565.00,2522.50,2543.40,174680,0
1,2015-04-06 00:00:00+05:30,2555.00,2560.44,2503.65,2546.00,60489,0
2,2015-04-07 00:00:00+05:30,2568.00,2592.00,2547.25,2586.10,51443,0
3,2015-04-08 00:00:00+05:30,2588.00,2652.00,2581.25,2644.35,54721,0
4,2015-04-09 00:00:00+05:30,2678.80,2683.95,2635.15,2661.15,67976,0
5,2015-04-10 00:00:00+05:30,2655.00,2669.65,2628.80,2651.95,37700,0
6,2015-04-13 00:00:00+05:30,2667.00,2667.00,2630.00,2650.25,37620,0
7,2015-04-15 00:00:00+05:30,2650.00,2712.00,2612.44,2624.70,140351,0
8,2015-04-16 00:00:00+05:30,2634.90,2634.90,2567.80,2585.19,115355,0
9,2015-04-17 00:00:00+05:30,2561.00,2561.00,2471.00,2476.19,374431,0


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83227,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83228,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83229,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83230,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,,0.00,0.0500,1,EQ,NSE,NSE
